# 사용자 정의 함수 

---
- 함수 기능 : 모델 학습 진행 함수 
- 함수 이름 : training 
- 매개변수  : 함수 구동 시 필요한 재료 => 학습을 위한 재료
    - 모델 인스턴스 
    - 학습 데이터셋 : 피쳐와 타겟 (학습 전 Tensor화)
    - 손실함수 인스턴스 
    - 최적화 인스턴스 
    - 학습 횟수 : 에포크 EPOCH 
    - 배치 크기 : BATCH_SIZE 
    - 배치 개수 : BTCH_CNT 
    - 검증용 데이터셋 : 피쳐와 타겟 (검증전 Tensor화)
- 함수 결과 : 학습 시 에포크당 손실값과 성능지표값, 검증의 손실값과 성능지표값 

* * * 
- 함수 기능 : 에포크 단위 모델 학습 진행 함수 
- 함수 이름 : epochTraining 
- 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료 
    - 모델 인스턴스 
    - 학습용 데이터셋 : 피쳐와 라벨 (학습 전 Tensor화)
    - 손실함수 인스턴스 
    - 최적화 인스턴스 
    - 배치크기 : BATCH_SIZE 
    - 배치개수 : BATCH_CNT
- 함수결과 : 손실값과 성능지표값 
* * *

In [5]:
# 모듈 로딩 
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optimizer
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

# 텐서 저장 및 실행 위치 설정 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# 에포크 당 학습 진행 후 손실값과 성능지표 값 반환 함수 

def epochTraining(model, feature, target, lossFunc, optimizer, batch_cnt, batch_size = 32, is_classifi = True):
    # - 에포크에서 배치크기 만큼 데이터셋 추출 후 학습 진행 

    loss_total = 0

    for batch in range(batch_cnt):
        # 배치 크기만큼 데이터셋 추출 및 텐서화 
        start = batch * batch_size  # [예 batch_size = 32 => 0, 32, 64, ....]
        end = start + batch_size    # [예 batch_size = 32 => 32, 64, 96, .......]

        x_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)

        # 학습 진행 
        pre_y = model(x_train).to(DEVICE)

        # 손실 계산 
        loss = lossFunc(pre_y, y_train)
        loss_total += loss 

        # 점수 추출 
        score = F1Score()(pre_y, y_train) if is_classifi else R2Score()(pre_y, y_train)
        score_total += score

        # 최적화 : w, b를 업데이트 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크당 손실값과 성능지표값 반환 
    # 테스트 및 검증 함수 사용 

    return loss_total / batch_cnt, score_total/batch_cnt

In [7]:
# 검증 및 테스트 진행 후  손실값과 성능지표 값 반환 함수 

def testing(model, feature, target, lossFunc):
    # - 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행 

    with torch.no_grad(): # .no_grad : 업데이트 기능 막음 

        # 배치 크기만큼 데이터셋 추출 및 텐서화
        feature_ts = torch.FloatTensor(feature.values).to(DEVICE)
        target_ts = torch.FloatTensor(target.values).to(DEVICE)

        # 학습 진행 
        pre_y = model(feature_ts).to(DEVICE) # feature 작업 많이 걸려서 gpu로 변경 

        # 손실 계산 
        loss = lossFunc(pre_y, target_ts)
        # loss_total += loss  # 한 개라서 더할 필요 x 

        # 점수 추출 
        score = F1Score()(pre_y, feature_ts)
        # score_total += score

   
   # 손실값과 성능지표값 반환 
    return loss, score

In [1]:
# 학습 진행 

def training():

    EPOCHS = 10
    TV_LOSS = {'TRAIN' : [], 'VAL':[]}
    TV_SCORE = {'TRAIN' : [], 'VAL':[]}

    for epoch in range(EPOCHS):
        # 학습 진행 
        train_loss, train_score = epochTraining()

        # 검증 진행 
        val_loss, val_score = testing()

        # 에포크당 학습 및 검증 결과 저장 
        TV_LOSS['TRAIN'].append(train_loss)
        TV_SCORE['TRAIN'].append(train_score)

        TV_LOSS['VAL'].append(val_loss)
        TV_SCORE['VAL'].append(val_score)

        # 에포크당 학습 및 검증 결과 출력 
        print(f'[{epoch}.{EPOCHS}] \n - [TRAIN] LOSS : {train_loss} SCORE : {train_score}')
        print(f'- [VALID] LOSS : {val_loss} SCORE : {val_score}')


단점 : 매개변수 다 넣어야 함 

장점 : 한 개 만들어두면 모든 모델 사용 ㅇ 